In [2]:
# ******** ACTIVATE VIRTUAL ENVIRONMENT ********
# ml_env\Scripts\Activate.ps1

# ******** MATH OPERATIONS ********
import numpy as np

# ******** DATA MANIPULATION ******
import pandas as pd

# ******** DATA VISUALIZATION ******
import seaborn as sns
import matplotlib.pyplot as plt

# ******** MATH & UTILITIES ********
from functools import partial
from collections import Counter

# ******** MACHINE LEARNING *******
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import minmax_scale
from sklearn.ensemble import RandomForestRegressor

In [3]:
def create_all_questions_prosodic_df(prosodic_df, score_df, interview_number):
    # Work on copies so the original DF isn't altered
    prosodic_df = prosodic_df.copy()
    score_df = score_df.copy()
    
    # Split and Clean Participant&Question column
    prosodic_df['Participant'] = prosodic_df['participant&question'].str.split('Q').str[0]
    prosodic_df['Participant'] = prosodic_df['Participant'].str.replace('^P+', '', regex=True, case=False)
    prosodic_df['Question'] = "Q" + prosodic_df['participant&question'].str.split('Q').str[1]
    prosodic_df['Question'] = prosodic_df['Question'].str.replace('^Q+', '', regex=True, case=False)
    
    # Clean Participant column in score_df
    score_df['Participant'] = score_df['Participant'].str.replace('^P+', '', regex=True, case=False)
        
    # Identify measurement columns
    exclude_cols = ['participant&question', 'Participant', 'Question']
    measurement_cols = [c for c in prosodic_df.columns if c not in exclude_cols]

    # Create new dataframe
    cleaned_df = pd.DataFrame()

    for participant in prosodic_df['Participant'].unique():
        chunk = prosodic_df[prosodic_df['Participant'] == participant]
        
        combined_data = {}
        for _, row in chunk.iterrows():
            question = row['Question']
            for col in measurement_cols:
                combined_data[f"{col}_{question}"] = row[col]
                combined_data = {k.strip(): v for k, v in combined_data.items()}

        combined_data['Participant'] = participant
        combined_data['Interview'] = interview_number
        temp_df = pd.DataFrame([combined_data])
        cleaned_df = pd.concat([cleaned_df, temp_df], ignore_index=True)
        
    # Merge with score dataframe
    cleaned_df = pd.merge(cleaned_df, score_df, on='Participant', how='left')
        
    return cleaned_df

def avg_questions_prosodic_df(prosodic_df, score_df, interview_number):
    # Work on copies so the original DF isn't altered
    prosodic_df = prosodic_df.copy()
    score_df = score_df.copy()

    # Split and Clean Participant&Question column
    prosodic_df['Participant'] = prosodic_df['participant&question'].str.split('Q').str[0]
    prosodic_df['Participant'] = prosodic_df['Participant'].str.replace('^P+', '', regex=True, case=False)
    prosodic_df['Question'] = "Q" + prosodic_df['participant&question'].str.split('Q').str[1]
    prosodic_df['Question'] = prosodic_df['Question'].str.replace('^Q+', '', regex=True, case=False)
    
    # Clean Participant column in score_df
    score_df['Participant'] = score_df['Participant'].str.replace('^P+', '', regex=True, case=False)
        
    # Identify measurement columns (everything except these)
    exclude_cols = ['participant&question', 'Participant', 'Question']
    measurement_cols = [c for c in prosodic_df.columns if c not in exclude_cols]

    # Convert measurement columns to numeric BEFORE averaging
    for col in measurement_cols:
        prosodic_df[col] = pd.to_numeric(prosodic_df[col], errors='coerce')

    # Create new dataframe
    cleaned_df = pd.DataFrame()
    
    for participant in prosodic_df['Participant'].unique():
        chunk = prosodic_df[prosodic_df['Participant'] == participant]
        
        combined_data = {}
        for col in measurement_cols:
            combined_data[f"avg_{col}"] = chunk[col].mean()

        combined_data['Participant'] = participant
        combined_data['Interview'] = interview_number  # ← IMPORTANT FIX
        temp_df = pd.DataFrame([combined_data])
        cleaned_df = pd.concat([cleaned_df, temp_df], ignore_index=True)
        
    # Merge with score dataframe
    cleaned_df = pd.merge(cleaned_df, score_df, on='Participant', how='left')
    
        
    return cleaned_df



# ---------------- Load in data ----------------
prosodic_features_df = pd.read_csv('../data/prosodic_features.csv')
score_df = pd.read_csv('../data/scores.csv')

# ---------------- Clean Columns ----------------
prosodic_features_df.columns = [col.strip() for col in prosodic_features_df.columns]
score_df.columns = [col.strip() for col in score_df.columns]

# --------------- Split into two dataframes ---------------
# One for first interview and one for second interview
prosodic_df_first = prosodic_features_df[prosodic_features_df['participant&question'].str.match(r'^P(?!P)', case=False)].copy()  # interview 1 (P only)
prosodic_df_second = prosodic_features_df[prosodic_features_df['participant&question'].str.match(r'^PP', case=False)].copy()     # interview 2 (PP only)
score_df_first = score_df[score_df['Participant'].str.match(r'^P(?!P)', case=False)].copy()        # interview 1 (P only)
score_df_second = score_df[score_df['Participant'].str.match(r'^PP', case=False)].copy()           # interview 2 (PP only)

#########################################################################
##### First dataframe: Each question's prosodic features as separate columns #####
#########################################################################

# -------------- Clean each dataframe separately ---------------
cleaned_df_first = create_all_questions_prosodic_df(prosodic_df_first, score_df_first, interview_number=1)
cleaned_df_second = create_all_questions_prosodic_df(prosodic_df_second, score_df_second, interview_number=2)
all_questions_df = pd.concat([cleaned_df_first, cleaned_df_second], ignore_index=True)

# -------------- Transform everything to numeric where possible ---------------
for col in all_questions_df.columns:
    all_questions_df[col] = pd.to_numeric(all_questions_df[col], errors='coerce')

# -------------- Sort by Participant and reset index ---------------
all_questions_df = all_questions_df.sort_values(by=['Participant']).reset_index(drop=True)

# -------------- Drop rows with any NaN values ---------------
all_questions_df = all_questions_df.dropna()

# --------------- Save cleaned prosodic features to new CSV ---------------
all_questions_df.to_csv('../data/all_q_cleaned_prosodic_features.csv', index=False)


#########################################################################
##### Second dataframe: Average prosodic features across questions #####
#########################################################################

# -------------- Clean each dataframe separately ---------------
avg_questions_df_first = avg_questions_prosodic_df(prosodic_df_first, score_df_first, interview_number=1)
avg_questions_df_second = avg_questions_prosodic_df(prosodic_df_second, score_df_second, interview_number=2)
avg_questions_df = pd.concat([avg_questions_df_first, avg_questions_df_second], ignore_index=True)

# -------------- Transform everything to numeric where possible ---------------
for col in avg_questions_df.columns:
    avg_questions_df[col] = pd.to_numeric(avg_questions_df[col], errors='coerce')
    
# -------------- Sort by Participant and reset index ---------------
avg_questions_df = avg_questions_df.sort_values(by=['Participant']).reset_index(drop=True)

# -------------- Drop rows with any NaN values ---------------
avg_questions_df = avg_questions_df.dropna()

# --------------- Save averaged cleaned prosodic features to new CSV ---------------
avg_questions_df.to_csv('../data/avg_cleaned_prosodic_features.csv', index=False)


## Prosodic Feature Selection

In [4]:
# Load Data Frames
average_df = pd.read_csv('../data/avg_cleaned_prosodic_features.csv')
all_df = pd.read_csv('../data/all_q_cleaned_prosodic_features.csv')

# #####################
# Print Correlation
# #####################

print_corr = False

# Find Correlated Pairs 
def get_top_corr_pairs(corr_matrix, top_n=5, most_positive=True):
    mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    corr_pairs = corr_matrix.where(mask).stack()
    sorted_corr = corr_pairs.sort_values(ascending=not most_positive)
    return sorted_corr.head(top_n)

# Correlation with Target Feature 
def print_corr_with_target(corr_matrix, target, top_n=5):
    series = corr_matrix[target].drop(labels=[target])
    print(f"\nTop features positively correlated with '{target}':")
    for feat, val in series.sort_values(ascending=False).head(top_n).items():
        print(f"{feat}: {val:.4f}")

    print(f"\nTop features negatively correlated with '{target}':")
    for feat, val in series.sort_values().head(top_n).items():
        print(f"{feat}: {val:.4f}")

if print_corr:
    corr_matrix = all_df.corr(method='pearson')
    print("\nCorrelation with Target Variable 'Overall':")
    print_corr_with_target(corr_matrix, target='Overall', top_n=5)
    print("\nCorrelation with Target Variable 'Excited':")
    print_corr_with_target(corr_matrix, target='Excited', top_n=5)


# ####################
# Test-Train Split
# ####################

# Averaged dataset test-train split (80-20)
X_average = average_df.drop(['Overall', 'Excited'], axis=1)
y_average = average_df[['Overall', 'Excited']]
X_train_avg, X_test_avg, y_train_avg, y_test_avg = train_test_split(X_average, y_average, test_size=0.20, random_state=42)

# Question concatenated dataset test-train split (80-20)
X_all = all_df.drop(['Overall', 'Excited'], axis=1)
y_all = all_df[['Overall', 'Excited']]
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_all, y_all, test_size=0.20, random_state=42)


# ####################
# Helper Functions
# ####################

#Mutual information function
mi = partial(mutual_info_regression, random_state=42, discrete_features=False)

#Grabs a vector of top-k selected features for each column of  y_train according to score_func
def get_selected_features(X_train, y_train, k, score_func):
    pipe_inner = Pipeline([
        ('kbst', SelectKBest(score_func, k=k)),
        ('regr', Ridge())
    ])
    pipe_outer = Pipeline([
        ('multi', MultiOutputRegressor(pipe_inner))
    ])
    pipe_outer.fit(X_train, y_train)

    # extract selected features per output
    feats_output_1 = pipe_outer.named_steps['multi'].estimators_[0].named_steps['kbst'].get_support(indices=True)
    feats_output_2 = pipe_outer.named_steps['multi'].estimators_[1].named_steps['kbst'].get_support(indices=True)

    return (
        X_train.columns[feats_output_1],
        X_train.columns[feats_output_2]
    )

#aggregates scoring for top-k selected features and selects features with highest average
def get_selected_features_aggregate(X_train, y_train, k, score_func):
    pipe_inner = Pipeline([
        ('kbst', SelectKBest(score_func, k=k)),
        ('regr', Ridge())
    ])
    pipe_outer = Pipeline([
        ('multi', MultiOutputRegressor(pipe_inner))
    ])
    pipe_outer.fit(X_train, y_train)

    scores_0 = pipe_outer.named_steps['multi'].estimators_[0].named_steps['kbst'].scores_
    scores_1 = pipe_outer.named_steps['multi'].estimators_[1].named_steps['kbst'].scores_
    scores = (np.nan_to_num(scores_0) + np.nan_to_num(scores_1)) / 2

    combined_score = minmax_scale(scores)
    top_k_idx = np.argsort(combined_score)[-k:][::-1]
    top_k_cols = list(X_train.columns[top_k_idx])

    return top_k_cols

#intersection function
def find_intersection(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    overlap_set = set1.intersection(set2)
    return list(overlap_set)

#union function
def find_union(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    total_set = set1.union(set2)
    return list(total_set)

### K-Best Feature Selection

In [5]:
# Storage for all feature sets per dataset
feature_sets = {
    "AVERAGED DATASET": [],
    "QUESTION-CONCATENATED DATASET": []
}

# Loop over both datasets
datasets = [
    ("AVERAGED DATASET", average_df),
    ("QUESTION-CONCATENATED DATASET", all_df)
]

# Print Nan counts for debugging
for col in datasets[1][1].columns:
    nan_count = datasets[1][1][col].isna().sum()
    if nan_count > 0:
        print(f"Column '{col}' has {nan_count} NaN values.")


for df_name, df in datasets:
    print(f"\nSelecting features from {df_name}")

    # Train/Test split
    X = df.drop(columns=['Overall', 'Excited'])
    y = df[['Overall', 'Excited']]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )

    # Feature selection loops
    for k in (5, 10, 15, 20, 25, 30):
        print(f"\n=== k = {k} ===")

        # Mutual Information
        mi_overall, mi_excited = get_selected_features(X_train, y_train, k, mi)
        mi_agg = get_selected_features_aggregate(X_train, y_train, k, mi)
        mi_inter = find_intersection(list(mi_overall), list(mi_excited))
        mi_union = find_union(list(mi_overall), list(mi_excited))

        # Random Forest
        rf_selected = X_train.columns[
            np.argsort(RandomForestRegressor(random_state=42)
                       .fit(X_train, y_train).feature_importances_)[-k:]
        ].tolist()

        # Append all feature sets as flat tuples
        feature_sets[df_name].extend([
            ("Selected Based on Overall", k, list(mi_overall)),
            ("Selected Based on Excited", k, list(mi_excited)),
            ("Intersection Overall & Excited", k, list(mi_inter)),
            ("Union Overall & Excited", k, list(mi_union)),
            ("Aggregate Overall & Excited", k, list(mi_agg)),
            ("Random Forest Selection", k, list(rf_selected))
        ])
        
        for feat_set, feat_names in [
            ("MI Overall", mi_overall),
            ("MI Excited", mi_excited),
            ("MI Intersection", mi_inter),
            ("MI Union", mi_union),
            ("MI Aggregate", mi_agg),
            ("RF Selected Features", rf_selected)
        ]:
            print(f"{feat_set}: {sorted(feat_names)}")
        


Selecting features from AVERAGED DATASET

=== k = 5 ===
MI Overall: ['avg_iInterval', 'avg_intensityMax', 'avg_numVoiceBreaks', 'avg_pitchUvsVRatio', 'avg_pitch_abs']
MI Excited: ['avg_PercentBreaks', 'avg_diffIntMaxMin', 'avg_intensityMax', 'avg_intensityMean', 'avg_intensityQuant']
MI Intersection: ['avg_intensityMax']
MI Union: ['avg_PercentBreaks', 'avg_diffIntMaxMin', 'avg_iInterval', 'avg_intensityMax', 'avg_intensityMean', 'avg_intensityQuant', 'avg_numVoiceBreaks', 'avg_pitchUvsVRatio', 'avg_pitch_abs']
MI Aggregate: ['avg_intensityMax', 'avg_intensityMean', 'avg_intensityQuant', 'avg_pitchUvsVRatio', 'avg_pitch_abs']
RF Selected Features: ['avg_avgBand3', 'avg_intensityMean', 'avg_intensityQuant', 'avg_pitchUvsVRatio', 'avg_pitch_abs']

=== k = 10 ===
MI Overall: ['avg_iInterval', 'avg_intensityMax', 'avg_numVoiceBreaks', 'avg_pitchUvsVRatio', 'avg_pitch_abs']
MI Excited: ['avg_PercentBreaks', 'avg_diffIntMaxMin', 'avg_intensityMax', 'avg_intensityMean', 'avg_intensityQuant']

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
import numpy as np

# All results df
total_results_df = pd.DataFrame()

# Output variables
output_vars = ['Overall', 'Excited']
datasets = [("AVERAGED DATASET", average_df), ("QUESTION-CONCATENATED DATASET", all_df)]


# ############################
# Process Averaged Dataset
# ############################
df_name, df = datasets[0]

print(f"\n==============================")
print(f"   DATASET: {df_name}")
print(f"==============================")

for feature_selector, k, feature_names in feature_sets[df_name]:
    for output_var in output_vars:
        print(f"\n--- Training for {output_var} using Feature selector '{feature_selector}' with {len(feature_names)} features = {feature_names} ---")

        # ##############
        # Prepare data
        # ##############
        
        print([col for col in df.columns])  # Convert generator to a list for printing
        
        X = df[feature_names]
        y = df[output_var]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # ####################
        # Decision Tree (CV)
        # ####################
        dt_grid = {
            "max_depth": [3, 5, 10, None],
            "min_samples_split": [2, 5, 10]
        }

        dt_model = GridSearchCV(
            DecisionTreeRegressor(random_state=42),
            dt_grid,
            cv=5,
            n_jobs=-1
        )

        dt_model.fit(X_train, y_train)
        dt_best = dt_model.best_estimator_

        dt_cv_score = dt_model.best_score_
        dt_test_score = dt_best.score(X_test, y_test)

        print(f"Decision Tree CV R^2:   {dt_cv_score:.4f}")
        print(f"Decision Tree Test R^2: {dt_test_score:.4f}")
        print(f"    Best Params: {dt_model.best_params_}")

        # ##########################
        # MLP (Deep Learning, CV)
        # ##########################
        mlp_pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("mlp", MLPRegressor(max_iter=1000, random_state=42))
        ])

        mlp_grid = {
            "mlp__hidden_layer_sizes": [(50,), (100,), (50, 50)],
            "mlp__activation": ["relu", "tanh"],
            "mlp__alpha": [0.0001, 0.001, 0.01]
        }

        mlp_model = GridSearchCV(
            mlp_pipeline,
            mlp_grid,
            cv=5,
            n_jobs=-1,
            error_score="raise"
        )

        mlp_model.fit(X_train, y_train)
        
        mlp_best = mlp_model.best_estimator_ 
        mlp_cv_score = mlp_model.best_score_
        mlp_test_score = mlp_best.score(X_test, y_test) 
        print(f"MLP CV R^2:   {mlp_cv_score:.4f}")
        print(f"MLP Test R^2: {mlp_test_score:.4f}")
        print(f"    Best Params: {mlp_model.best_params_}")
        
        results_df = pd.DataFrame({
            'Dataset': [df_name],
            'Feature_Selector': [feature_selector],
            'Num_Features': [len(feature_names)],
            'Output_Variable': [output_var],
            'DT_CV_R2': [dt_cv_score],
            'DT_Test_R2': [dt_test_score],
            'DT_Best_Params': [dt_model.best_params_],
            'MLP_CV_R2': [mlp_cv_score],
            'MLP_Test_R2': [mlp_test_score],
            'MLP_Best_Params': [mlp_model.best_params_]
        })

        total_results_df = pd.concat([total_results_df, results_df], ignore_index=True)
        

        
# ############################
# Process Question-Concatenated Dataset
# ############################

df_name, df = datasets[1]


print(f"\n==============================")
print(f"   DATASET: {df_name}")
print(f"==============================")

for feature_selector, k, feature_names in feature_sets[df_name]:
    for output_var in output_vars:
        print(f"\n--- Training for {output_var} using Feature selector '{feature_selector}' with {len(feature_names)} features = {feature_names} ---")

        # ##############
        # Prepare data
        # ##############
        
        print([col for col in df.columns])  # Convert generator to a list for printing
        
        X = df[feature_names]
        y = df[output_var]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # ####################
        # Decision Tree (CV)
        # ####################
        dt_grid = {
            "max_depth": [3, 5, 10, None],
            "min_samples_split": [2, 5, 10]
        }

        dt_model = GridSearchCV(
            DecisionTreeRegressor(random_state=42),
            dt_grid,
            cv=5,
            n_jobs=-1
        )

        dt_model.fit(X_train, y_train)
        dt_best = dt_model.best_estimator_

        dt_cv_score = dt_model.best_score_
        dt_test_score = dt_best.score(X_test, y_test)

        print(f"Decision Tree CV R^2:   {dt_cv_score:.4f}")
        print(f"Decision Tree Test R^2: {dt_test_score:.4f}")
        print(f"    Best Params: {dt_model.best_params_}")

        # ##########################
        # MLP (Deep Learning, CV)
        # ##########################
        mlp_pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("mlp", MLPRegressor(max_iter=1000, random_state=42))
        ])

        mlp_grid = {
            "mlp__hidden_layer_sizes": [(50,), (100,), (50, 50)],
            "mlp__activation": ["relu", "tanh"],
            "mlp__alpha": [0.0001, 0.001, 0.01]
        }

        mlp_model = GridSearchCV(
            mlp_pipeline,
            mlp_grid,
            cv=5,
            n_jobs=-1,
            error_score="raise"
        )
        mlp_model.fit(X_train, y_train)
        mlp_best = mlp_model.best_estimator_
        mlp_cv_score = mlp_model.best_score_
        mlp_test_score = mlp_best.score(X_test, y_test)
        print(f"MLP CV R^2:   {mlp_cv_score:.4f}")
        print(f"MLP Test R^2: {mlp_test_score:.4f}")
        print(f"    Best Params: {mlp_model.best_params_}")
        
        results_df = pd.DataFrame({
            'Dataset': [df_name],
            'Feature_Selector': [feature_selector],
            'Num_Features': [len(feature_names)],
            'Output_Variable': [output_var],
            'DT_CV_R2': [dt_cv_score],
            'DT_Test_R2': [dt_test_score],
            'DT_Best_Params': [dt_model.best_params_],
            'MLP_CV_R2': [mlp_cv_score],
            'MLP_Test_R2': [mlp_test_score],
            'MLP_Best_Params': [mlp_model.best_params_]
        })

        total_results_df = pd.concat([total_results_df, results_df], ignore_index=True)
        
total_results_df.to_csv('../data/model_performance_results.csv', index=False)


   DATASET: AVERAGED DATASET

--- Training for Overall using Feature selector 'Selected Based on Overall' with 5 features = ['avg_pitch_abs', 'avg_pitchUvsVRatio', 'avg_intensityMax', 'avg_numVoiceBreaks', 'avg_iInterval'] ---
['avg_duration', 'avg_energy', 'avg_min_pitch', 'avg_max_pitch', 'avg_mean_pitch', 'avg_pitch_sd', 'avg_pitch_abs', 'avg_pitch_quant', 'avg_pitchUvsVRatio', 'avg_diffPitchMaxMin', 'avg_diffPitchMaxMean', 'avg_diffPitchMaxMode', 'avg_intensityMin', 'avg_intensityMax', 'avg_intensityMean', 'avg_intensitySD', 'avg_intensityQuant', 'avg_diffIntMaxMin', 'avg_diffIntMaxMean', 'avg_diffIntMaxMode', 'avg_avgVal1', 'avg_avgBand2', 'avg_avgBand3', 'avg_jitter', 'avg_shimmer', 'avg_numVoiceBreaks', 'avg_PercentBreaks', 'avg_speakRate', 'avg_numPause', 'avg_maxDurPause', 'avg_avgDurPause', 'avg_TotDurPause:3', 'avg_iInterval', 'avg_MaxRising:3', 'avg_MaxFalling:3', 'Participant', 'Interview', 'Overall', 'Excited']
Decision Tree CV R^2:   -0.0993
Decision Tree Test R^2: 0.17

In [18]:
# total_results_df.sort_values(by=['MLP_Test_R2'], ascending=False).head(10)

total_results_df.sort_values(by=['DT_Test_R2'], ascending=False).head(10)


,Dataset,Feature_Selector,Num_Features,Output_Variable,DT_CV_R2,DT_Test_R2,DT_Best_Params,MLP_CV_R2,MLP_Test_R2,MLP_Best_Params
1,AVERAGED DATASET,Selected Based on Overall,5,Excited,0.311747,0.524344,"{'max_depth': 5, 'min_samples_split': 2}",0.433981,0.524401,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
131,QUESTION-CONCATENATED DATASET,Random Forest Selection,25,Excited,-0.035058,0.508913,"{'max_depth': 3, 'min_samples_split': 5}",-0.349716,-0.007451,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
119,QUESTION-CONCATENATED DATASET,Random Forest Selection,20,Excited,-0.028451,0.508913,"{'max_depth': 3, 'min_samples_split': 2}",-0.046356,0.178721,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
143,QUESTION-CONCATENATED DATASET,Random Forest Selection,30,Excited,0.137651,0.508913,"{'max_depth': 3, 'min_samples_split': 2}",-1.163301,0.024384,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00..."
127,QUESTION-CONCATENATED DATASET,Union Overall & Excited,37,Excited,0.137541,0.498907,"{'max_depth': 3, 'min_samples_split': 2}",-0.857385,0.208519,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
95,QUESTION-CONCATENATED DATASET,Random Forest Selection,10,Excited,0.103637,0.489332,"{'max_depth': 3, 'min_samples_split': 5}",0.464255,0.319231,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
2,AVERAGED DATASET,Selected Based on Excited,5,Overall,-0.002673,0.459830,"{'max_depth': 3, 'min_samples_split': 10}",0.161630,0.443521,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.00..."
123,QUESTION-CONCATENATED DATASET,Selected Based on Excited,25,Excited,0.167176,0.443716,"{'max_depth': 3, 'min_samples_split': 10}",-0.117389,0.354448,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
115,QUESTION-CONCATENATED DATASET,Union Overall & Excited,31,Excited,0.158063,0.436661,"{'max_depth': 3, 'min_samples_split': 5}",-0.514921,0.107735,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
13,AVERAGED DATASET,Selected Based on Overall,10,Excited,0.208317,0.420585,"{'max_depth': 3, 'min_samples_split': 2}",0.383037,0.495530,"{'mlp__activation': 'tanh', 'mlp__alpha': 0.01..."
